<a href="https://colab.research.google.com/github/ekayulianto/ANN/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Importing the dataset

Import Data From Local

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Churn_Modelling.csv to Churn_Modelling (4).csv


In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
dataset.shape

(10000, 14)

Drop Features

In [ ]:
dataset = dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Handling Missing Value

In [ ]:
import missingno as msno

In [ ]:
def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
miss_values = null_values(dataset)
miss_values

Dataframe has 11 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


### Feature Engeneering (Label Encoding)

1. Label Encoding the "Gender" column

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [ ]:
dataset.Gender.value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [ ]:
def gender_(a):
    value=1
    tampung = str(a)
    if tampung=='Female':
        value=0
    else:
        value=1
    return value

In [ ]:
dataset['Gender_Flag'] = dataset["Gender"].apply(lambda x: gender_(x))
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [ ]:
dataset = dataset.drop(['Gender'], axis=1)
dataset.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag
0,619,France,42,2,0.00,1,1,1,101348.88,1,0
1,608,Spain,41,1,83807.86,1,0,1,112542.58,0,0
2,502,France,42,8,159660.80,3,1,0,113931.57,1,0
3,699,France,39,1,0.00,2,0,0,93826.63,0,0
4,850,Spain,43,2,125510.82,1,1,1,79084.10,0,0


2. One Hot Encoding the "Geography" column

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Age              10000 non-null  int64  
 3   Tenure           10000 non-null  int64  
 4   Balance          10000 non-null  float64
 5   NumOfProducts    10000 non-null  int64  
 6   HasCrCard        10000 non-null  int64  
 7   IsActiveMember   10000 non-null  int64  
 8   EstimatedSalary  10000 non-null  float64
 9   Exited           10000 non-null  int64  
 10  Gender_Flag      10000 non-null  int64  
dtypes: float64(2), int64(8), object(1)
memory usage: 859.5+ KB


Split Categorical and Numerical Data


In [ ]:
num_features = dataset.select_dtypes(include=["int64", "float64"])           #mengambil data dengan fiturnya(tipe) yaitu int, dan float
num_features.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag
0,619,42,2,0.00,1,1,1,101348.88,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0


In [ ]:
num_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Age              10000 non-null  int64  
 2   Tenure           10000 non-null  int64  
 3   Balance          10000 non-null  float64
 4   NumOfProducts    10000 non-null  int64  
 5   HasCrCard        10000 non-null  int64  
 6   IsActiveMember   10000 non-null  int64  
 7   EstimatedSalary  10000 non-null  float64
 8   Exited           10000 non-null  int64  
 9   Gender_Flag      10000 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 781.4 KB


In [ ]:
cat_features = dataset.select_dtypes(include=["object"])           #mengambil data dengan fiturnya(tipe) yaitu object
cat_features.head()

,Geography
0,France
1,Spain
2,France
3,France
4,Spain


In [ ]:
cat_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Geography  10000 non-null  object
dtypes: object(1)
memory usage: 78.2+ KB


In [ ]:
cat_features.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [ ]:
cat_features.head(10)

,Geography
0,France
1,Spain
2,France
3,France
4,Spain
5,Spain
6,France
7,Germany
8,France
9,France


In [ ]:
## One-Hot Encoding
cat_featureohe=pd.get_dummies(cat_features,columns=cat_features.columns.values)
cat_featureohe.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1


Merge Data

In [ ]:
df = num_features.merge(cat_featureohe, left_index=True, right_index=True)   #join dummies dan num_features
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  int64  
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Gender_Flag        10000 non-null  int64  
 10  Geography_France   10000 non-null  uint8  
 11  Geography_Germany  10000 non-null  uint8  
 12  Geography_Spain    10000 non-null  uint8  
dtypes: float64(2), int64(8), uint8(3)
memory usage: 810.7 KB


### Splitting the dataset into the Training set and Test set

Split Data

In [ ]:
X = df.drop('Exited', axis=1)       ## Independetn Var
y = df['Exited']                    ## Dependent Var (target)

In [ ]:
## Train-Test Split

from sklearn.model_selection import train_test_split

seed = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)         #menghitung jumlah status(y) pd data training
dict(zip(unique, counts))

{0: 6368, 1: 1632}

### Feature Scaling (Standar Scaller)

In [ ]:
X_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Flag,Geography_France,Geography_Germany,Geography_Spain
7389,667,34,5,0.00,2,1,0,163830.64,0,0,0,1
9275,427,42,1,75681.52,1,1,1,57098.00,1,0,1,0
2995,535,29,2,112367.34,1,1,0,185630.76,0,1,0,0
5316,654,40,5,105683.63,1,1,0,173617.09,1,0,0,1
356,850,57,8,126776.30,2,1,1,132298.49,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9225,594,32,4,120074.97,2,1,1,162961.79,0,0,1,0
4859,794,22,4,114440.24,1,1,1,107753.07,0,0,0,1
3264,738,35,5,161274.05,2,1,0,181429.87,1,1,0,0
9845,590,38,9,0.00,2,1,1,148750.16,0,0,0,1


In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train_std = ss.fit_transform(X_train)           ## Standard Scaller of X_train
X_test_std = ss.fit_transform(X_test)             ## Standard Scaller of X_test

In [ ]:
X_train_std

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-2.30455945,  0.30102557, -1.37744033, ..., -1.01460667,
         1.75486502, -0.57369368],
       [-1.19119591, -0.94312892, -1.031415  , ...,  0.98560362,
        -0.5698444 , -0.57369368],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       [-0.62420521, -0.08179119,  1.39076231, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-0.28401079,  0.87525072, -1.37744033, ..., -1.01460667,
         1.75486502, -0.57369368]])

In [ ]:
X_test_std

array([[-0.56129438, -0.39401698,  0.9869706 , ..., -0.95692675,
         1.62776996, -0.57427105],
       [-1.33847768,  0.07611425, -1.08432132, ...,  1.04501206,
        -0.61433742, -0.57427105],
       [ 0.58347561,  0.26416674,  0.9869706 , ..., -0.95692675,
        -0.61433742,  1.74133801],
       ...,
       [-0.76084144, -0.29999074, -1.42953664, ..., -0.95692675,
        -0.61433742,  1.74133801],
       [-0.0046631 , -0.48804323, -0.39389068, ..., -0.95692675,
         1.62776996, -0.57427105],
       [-0.81335383, -0.86414821,  0.9869706 , ..., -0.95692675,
         1.62776996, -0.57427105]])

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()            ## From Tensorflow

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))          ## NOte: unit(neuron of hidden layer), activation (fungsi aktivasi), bias (bias)

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))          ## NOte: unit(neuron of hidden layer), activation (fungsi aktivasi), bias (bias)

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))          ## NOte: unit = 1 (Karena, outputnya erupa binary, 1 atau 0). kalau output non-binary klasifikasi (ex: A, B, C) maka dapat dibuat 3 buah neuron
## fungsi aktivasi yang digunakan yaitu sigmoid, karena bagus untuk predict binary (1 atau 0)
## jika output non binary classification(ex: 3 class), maka activation='softmax'(fungsi aktivasi pada hidden to output yaitu "softmax")

## Part 3 - Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'] )   
## optimizer(adam) merupakan kombinasi RMSprop dan Stochastic Gradient Descent dengan momentum
## loss (binary_crossentropy), karena outputnya berupa binary classification
## jika output non-binary (ex: 3 class), maka loss = 'categorical_crossentropy'
## metrics yg dimunculkan yaitu akurasi

### Training the ANN on the Training set

In [ ]:
len (y_train)

8000

In [ ]:
ann.fit(X_train_std, y_train, batch_size= 32, epochs=100)                ## epoch =  jumlah iterasi, dan mengg
## batch size yaitu mengambil jumlah sample untuk ANN melakukan training set. 
## Contoh terdapat 1050 training data, dan batch_size=100, maka algoritma akan mengambil 100 sample pertama dari training data, dan mengambil 100 keduan, dst.

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5570 - accuracy: 0.7345
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4638 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4436 - accuracy: 0.7981
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.8045
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.8073
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4257 - accuracy: 0.8098
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4224 - accuracy: 0.8104
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4194 - accuracy: 0.8127
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4159 - accuracy: 0.8160
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4126 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Single Test**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [ ]:
print(ann.predict(ss.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))       ## NOte: 3 angka pertama (1,0,0) merupakan isi dari one hot encodingnya 'France'
## untuk normalisasinya (new data/tes) hanya menggunakan transform

[[0.]]


Therefore, our ANN model predicts that this customer stays in the bank!


In [ ]:
print(ann.predict(ss.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)             ## 0.5 is default threshold of probability

[[False]]


Customer still stay in bank, so we can't say goodbay to this customer

### Predicting the Test set results

Predict (DataFrame with index)

In [ ]:
proba = ann.predict(X_test_std)
proba

array([[0.2775933 ],
       [0.3619659 ],
       [0.18783751],
       ...,
       [0.06088361],
       [0.23701262],
       [0.20256442]], dtype=float32)

In [ ]:
pred_test = {'pred_prob_topay': proba[:,0]}

result = pd.DataFrame(pred_test)           ## hasil probabilitasnya

hasil_akhir = df.loc[X_test.index].merge(result, left_index=True, right_index=True)
hasil_akhir.head(100)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag,Geography_France,Geography_Germany,Geography_Spain,pred_prob_topay
898,523,40,2,102967.41,1,1,0,128702.10,1,0,1,0,0,0.026588
1087,498,31,10,0.00,2,1,0,13892.57,0,0,1,0,0,0.097338
1819,714,45,8,150900.29,2,0,1,139889.15,0,1,0,0,1,0.219312
382,492,30,10,77168.87,2,0,1,146700.22,0,0,0,1,0,0.242121
580,701,36,8,0.00,2,1,0,169161.46,0,1,0,0,1,0.638579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,694,37,3,0.00,2,1,1,147012.22,0,0,0,0,1,0.520181
1761,662,36,4,166909.20,2,1,0,138871.12,1,0,1,0,0,0.011397
1835,696,38,6,142316.14,1,1,1,8018.49,0,1,0,1,0,0.011796
119,699,34,4,185173.81,2,1,0,120834.48,0,1,0,1,0,0.006548


In [ ]:
hasil_akhir['prediction'] = hasil_akhir['pred_prob_topay'].apply(lambda x: 1 if x > 0.5 else 0)
hasil_akhir.head(200)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag,Geography_France,Geography_Germany,Geography_Spain,pred_prob_topay,prediction
898,523,40,2,102967.41,1,1,0,128702.10,1,0,1,0,0,0.026588,0
1087,498,31,10,0.00,2,1,0,13892.57,0,0,1,0,0,0.097338,0
1819,714,45,8,150900.29,2,0,1,139889.15,0,1,0,0,1,0.219312,0
382,492,30,10,77168.87,2,0,1,146700.22,0,0,0,1,0,0.242121,0
580,701,36,8,0.00,2,1,0,169161.46,0,1,0,0,1,0.638579,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,538,49,9,141434.04,1,0,0,173779.25,1,0,0,0,1,0.016661,0
1842,745,35,8,0.00,2,1,1,116581.10,0,0,0,0,1,0.387403,0
519,707,58,6,89685.92,1,0,1,126471.13,0,1,1,0,0,0.116431,0
558,741,53,3,0.00,2,1,1,38913.68,0,1,0,0,1,0.396118,0


In [ ]:
hasil_akhir['actual'] = y_test
hasil_akhir.head(200)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Flag,Geography_France,Geography_Germany,Geography_Spain,pred_prob_topay,prediction,actual
898,523,40,2,102967.41,1,1,0,128702.10,1,0,1,0,0,0.026588,0,1
1087,498,31,10,0.00,2,1,0,13892.57,0,0,1,0,0,0.097338,0,0
1819,714,45,8,150900.29,2,0,1,139889.15,0,1,0,0,1,0.219312,0,0
382,492,30,10,77168.87,2,0,1,146700.22,0,0,0,1,0,0.242121,0,0
580,701,36,8,0.00,2,1,0,169161.46,0,1,0,0,1,0.638579,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,538,49,9,141434.04,1,0,0,173779.25,1,0,0,0,1,0.016661,0,1
1842,745,35,8,0.00,2,1,1,116581.10,0,0,0,0,1,0.387403,0,0
519,707,58,6,89685.92,1,0,1,126471.13,0,1,1,0,0,0.116431,0,0
558,741,53,3,0.00,2,1,1,38913.68,0,1,0,0,1,0.396118,0,0


In [ ]:
## komparasi predict dan actual

compare = hasil_akhir[['prediction', 'actual']]
compare

,prediction,actual
898,0,1
1087,0,0
1819,0,0
382,0,0
580,1,0
...,...,...
1712,0,0
1578,0,0
1624,0,0
1774,0,0


Result (Numpy)

In [ ]:
y_pred = ann.predict(X_test_std)
y_pred = (y_pred>0.5)

In [ ]:
y_pred[]

array([False])

In [ ]:
y_test.values.reshape(len(y_test), 1)

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.values.reshape(len(y_test), 1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [ ]:
from sklearn.metrics  import confusion_matrix, accuracy_score
cm  = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[1518   77]
 [ 202  203]]


0.8605

Correct Prediction : 
1. Customer stay in bank = 1518 (True Positive)
2. Customer leaves the bank = 203 (True Negative)

Incorect Prediction
1. Customer stay in bank = 202 (False Positive)
2. Customer Leaves the bank = 77 (False Negative)